# MPI messaging

This script will analyse `broadcast` and `barrier` messging, each with 3 different algorithms.


In [1000]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import plotly.express as px
import plotly.graph_objects as go

path = os.getcwd()

In [1001]:
# Data gathered using the latency test within 1 socket
latency_by_size = pd.DataFrame(columns=['Start_CPU', 'End_CPU', 'MessageSize', 'Latency'], 
								data= [
										[0, 2, 1, 		0.19], 
										[0, 2, 2, 		0.19], 
										[0, 2, 4, 		0.19], 
										[0, 2, 8, 		0.19],
										[0, 2, 16, 		0.19],
										[0, 2, 32, 		0.24],
										[0, 2, 64, 		0.24],
										[0, 2, 128, 	0.33],
										[0, 2, 256, 	0.37],
										[0, 2, 512, 	0.44],
										[0, 2, 1024, 	0.54],
										[0, 2, 2048, 	0.75],
										[0, 2, 4096, 	1.21],
										[0, 2, 8192, 	2.04],
										[0, 2, 16384, 	3.15],
										[0, 2, 32768, 	4.97],
										[0, 2, 65536, 	8.40],
										[0, 2, 131072, 	14.95],
										[0, 2, 262144, 	16.22],
										[0, 2, 524288, 	30.01],
										[0, 2, 1048576, 61.65],
										[0, 2, 2097152, 155.70],
										[0, 2, 4194304, 356.84]
									]
								)

In [1002]:

def plot_latency_vs_processes(df, algorithm, scale):
	'''
	Plots the latency vs the number of processes for the broadcast operation
	'''

	# Initialize the figure
	fig = go.Figure()

	# Start plotting the data
	for selected in range(len(df)):
		# Select the data
		used_df = df[selected]
		# Select the message sizes
		for message_size in selected_sizes:
			used_size_df = used_df[used_df['MessageSize']==message_size]
			# Plot the data
			fig.add_trace(go.Scatter(x=used_size_df['Processes'], 
									y=used_size_df['Latency'],
									mode='lines+markers', name=str(message_size)))

			# Set the parameter alpha as the one experimentally measured for in-socekt communication
			alpha = latency_by_size[latency_by_size['MessageSize']==message_size]['Latency'].values[0]

			# Select the theoretical model based on the algorithm
			if algorithm == 'Default':
			# CPU frequency is 3.7 GHz,
				# Unit is in microseconds, so a 10**6 is needed
				# 5 is asspullery ( expected 64 bytes per cycle, but that's the best case scenario; probably lower)
				beta  = 10**6 * (1 / ((3.7 * 10**9 )*5))
				beta2 = 10**6 * (1 / ((3.7 * 10**9 )*5))

				# Differentiate for sizes smaller than 32KB and larger than 32KB, which is the size of the L1 cache
				if message_size <= 32384:
					fig.add_trace(go.Scatter(x=used_size_df['Processes'],

											# The actual model
											y = alpha/2 + beta * message_size * ( used_size_df['Processes'] - 1 ),


											mode='lines',
											name=str(message_size)+'Theoretical Latency for size ',
											line=dict(color='black', dash='dash')))
				else:
					fig.add_trace(go.Scatter(x=used_size_df['Processes'],

											# The actual model
											y = alpha/2 + beta * message_size * ( used_size_df['Processes'] - 1 ),


											mode='lines',
											name=str(message_size)+'Theoretical Latency for size ',
											line=dict(color='red', dash='dash')))
			elif algorithm == 'binary tree':
				# Set the beta parameter for the theoretical model 
				# This will need to change for the default algorithm, currently working with the chain algorithm
				beta  = 10**6 * (1 / ((3.7 * 10**9 )*15))
				beta2 = 10**6 * (1 / ((3.7 * 10**9 )*9))

				out = np.ceil(np.log2(((used_size_df['Processes'][0:11]))))*(alpha/2 ) + beta * message_size * (used_size_df['Processes'][0:11])
				out2 = np.ceil(np.log2(((used_size_df['Processes'][12:]))))*(alpha/2 ) + beta2 * message_size * (used_size_df['Processes'][12:])
				out = pd.concat([out, out2], ignore_index=True)

				# Differentiate for sizes smaller than 32KB and larger than 32KB, which is the size of the L1 cache
				if message_size <= 32384:
					fig.add_trace(go.Scatter(x=used_size_df['Processes'],

											# The actual model
											# y = alpha/2 + beta * message_size * ( used_df['Processes'] - 1 ),
											y = out,

											mode='lines',
											name=str(message_size)+'Theoretical Latency for size ',
											line=dict(color='black', dash='dash')))

				else:
					fig.add_trace(go.Scatter(x=used_size_df['Processes'],

											# The actual model
											y = alpha/2 + beta * message_size * ( used_size_df['Processes'] - 1 ),


											mode='lines',
											name=str(message_size)+'Theoretical Latency for size ',
											line=dict(color='red', dash='dash')))

			elif algorithm == 'chain':
				# CPU frequency is 3.7 GHz,
				# Unit is in microseconds, so a 10**6 is needed
				# 5 is asspullery ( expected 64 bytes per cycle, but that's the best case scenario; probably lower)
				beta  = 10**6 * (1 / ((3.7 * 10**9 )*5))
				beta2 = 10**6 * (1 / ((3.7 * 10**9 )*5))

				# Differentiate for sizes smaller than 32KB and larger than 32KB, which is the size of the L1 cache
				if message_size <= 32384:
					fig.add_trace(go.Scatter(x=used_size_df['Processes'],

											# The actual model: the pure latency for the size, halved because it's
											# a broadcast operation, then the number of pocesses -1 because it's the
											# number of messages that need to be sent
											y = alpha/2 + beta * message_size * ( used_size_df['Processes'] - 1 ),


											mode='lines',
											name=str(message_size)+'Theoretical Latency for size ',
											line=dict(color='black', dash='dash')))
				else:
					fig.add_trace(go.Scatter(x=used_size_df['Processes'],

											# The actual model: the pure latency for the size, halved because it's
											# a broadcast operation, then the number of pocesses -1 because it's the
											# number of messages that need to be sent
											y = alpha/2 + beta * message_size * ( used_size_df['Processes'] - 1 ),


											mode='lines',
											name=str(message_size)+'Theoretical Latency for size ',
											line=dict(color='red', dash='dash')))
  

	# Set the layout of the figure, title, axis labels, etc.
	# Select the scale type of the y axis
	fig.update_layout(title='Latency vs Processes for the broadcast using '+algorithm+' Algorithm', 
						xaxis_title='Processes', 
						yaxis_title='Latency (us)', 
						legend_title='Message Size',
						xaxis=dict(tickvals=tickvals),
						yaxis=dict(type=scale)
					)
	fig.show()



## `broadcast` messaging

Let's first load the data

In [1003]:
# Set path to the data
bcast_path = path + '/bcast/results_bcast/'  # Replace with your actual path

# Get the list of files in the directory
files = os.listdir(bcast_path)

# Create a list to store the dataframes
dfs = []

# Read the data from the files; for each file, check if 
# the name contains binary, default or chain and set the
# corresponding value in the dataframe

for file in files:
    if 'binary' in file:
        df = pd.read_csv(os.path.join(bcast_path, file))
        df['algorithm'] = 'Binary Tree'
        dfs.append(df)
    elif 'default' in file:
        df = pd.read_csv(os.path.join(bcast_path, file))
        df['algorithm'] = 'Default'
        dfs.append(df)
    elif 'chain' in file:
        df = pd.read_csv(os.path.join(bcast_path, file))
        df['algorithm'] = 'Chain'
        dfs.append(df)

# Filter out dataframes where the 'Algorithm' column contains '${algorithm}' or is null
valid_dfs = []
for df in dfs:
    if 'Algorithm' in df.columns:
        df['Algorithm'] = df['Algorithm'].astype(str)
        if not df['Algorithm'].str.contains(r'\$\{algorithm\}').any():
            valid_dfs.append(df)

# Split the data into separate dataframes for each algorithm
default_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'default']
binary_tree_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'binary_tree']
chain_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'chain']

# Select an allocation strategy
default_df = [df[df['Allocation']=='core'] for df in default_df]
binary_tree_df = [df[df['Allocation']=='core'] for df in binary_tree_df]
chain_df = [df[df['Allocation']=='core'] for df in chain_df]

# Select the message sizes and other plot parameters
message_sizes = default_df[0]['MessageSize'].unique()
allocations = default_df[0]['Allocation'].unique()
tickvals = [2, 6, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48]
selected_sizes = [8192] #16384, 524288, 1048576]

binary_tree_df[0]

,Algorithm,Allocation,Processes,MessageSize,Latency,algorithm
0,binary_tree,core,2,1,0.20,Binary Tree
1,binary_tree,core,2,2,0.21,Binary Tree
2,binary_tree,core,2,4,0.21,Binary Tree
3,binary_tree,core,2,8,0.19,Binary Tree
4,binary_tree,core,2,16,0.20,Binary Tree
...,...,...,...,...,...,...
982,binary_tree,core,48,65536,102.78,Binary Tree
983,binary_tree,core,48,131072,227.99,Binary Tree
984,binary_tree,core,48,262144,444.73,Binary Tree
985,binary_tree,core,48,524288,885.89,Binary Tree


## Default algorithm

Let's plot the default algorithm

In [1004]:
plot_latency_vs_processes(default_df, 'Default', 'linear')
plot_latency_vs_processes(default_df, 'Default', 'log')

## Binary tree algorithm



In [1005]:
plot_latency_vs_processes(binary_tree_df, 'binary tree', 'linear')
plot_latency_vs_processes(binary_tree_df, 'binary tree','log')

### Chain algorithm

Latency Model:  
$T = \alpha + \beta n $  
Where:

* $\alpha$ is the latency (startup time) per message.
* $\beta$ is the time per byte.
* $n$ is the number of bytes in the message.

Thus, I expect the total time dependendency on CPUs ($np$) to be:  
$T = \alpha + np(\alpha + \beta n )$  

In [1006]:
plot_latency_vs_processes(chain_df, 'chain', 'linear')
plot_latency_vs_processes(chain_df, 'chain','log')

## `barrier` messaging

Lore ipsum etc etc

In [1007]:
# Set path to the data
barrier_path = path + '/barrier/results_barrier/'  # Replace with your actual path

# Get the list of files in the directory
files = os.listdir(barrier_path)

# Create a list to store the dataframes
dfs = []
default_df = []
tree_df = []
linear_df = []

# Read the data from the files; for each file, check if 
# the name contains binary, default or chain and set the
# corresponding value in the dataframe

for file in files:
    if 'tree' in file:
        df = pd.read_csv(os.path.join(barrier_path, file))
        df['algorithm'] = 'Tree'
        dfs.append(df)
    elif 'default' in file:
        df = pd.read_csv(os.path.join(barrier_path, file))
        df['algorithm'] = 'Default'
        dfs.append(df)
    elif 'linear' in file:
        df = pd.read_csv(os.path.join(barrier_path, file))
        df['algorithm'] = 'Linear'
        dfs.append(df)

# Filter out dataframes where the 'Algorithm' column contains '${algorithm}' or is null
valid_dfs = []
for df in dfs:
    if 'Algorithm' in df.columns:
        df['Algorithm'] = df['Algorithm'].astype(str)
        if not df['Algorithm'].str.contains(r'\$\{algorithm\}').any():
            valid_dfs.append(df)

# Split the data into separate dataframes for each algorithm
default_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'default']
tree_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'tree']
linear_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'linear']

# Select an allocation strategy
default_df = [df[df['Allocation']=='core'] for df in default_df]
tree_df = [df[df['Allocation']=='core'] for df in tree_df]
linear_df = [df[df['Allocation']=='core'] for df in linear_df]

allocations = default_df[0]['Allocation'].unique()


# Initialize the figure
fig = go.Figure()

# Start plotting the data
for selected in range(len(default_df)):
	# The two runs
	used_df = default_df[selected]
	# Only plot for 512KB and 1MB, a.k.a. one fitting entirely in L1 and one not fitting
	# Plot the data
	fig.add_trace(go.Scatter(x=used_df['Processes'], 
							y=used_df['Latency'],
							mode='lines+markers',
							name="Run "+str(selected)))
# Update the layout by naming the appropirate tings and highlighting some places of interest in the x-axis
fig.update_layout(title='Latency vs Processes for the barrier using the Default Algorithm', 
					xaxis_title='Processes', 
					yaxis_title='Latency (us)', 
					xaxis=dict(tickvals=tickvals)
				)

fig.show()

In [1008]:
# Initialize the figure
fig = go.Figure()

tree_df = [tree_df[i] for i in range(len(tree_df)) if i !=5]

# Start plotting the data
for selected in range(len(tree_df)):
	# The two runs
	used_df = tree_df[selected]
	# Only plot for 512KB and 1MB, a.k.a. one fitting entirely in L1 and one not fitting
	# Plot the data
	fig.add_trace(go.Scatter(x=used_df['Processes'], 
							y=used_df['Latency'],
							mode='lines+markers',
							name="Run "+str(selected)))
# Update the layout by naming the appropirate tings and highlighting some places of interest in the x-axis
fig.update_layout(title='Latency vs Processes for the barrier using the the tree Algorithm', 
					xaxis_title='Processes', 
					yaxis_title='Latency (us)', 
					xaxis=dict(tickvals=tickvals)
				)

fig.show()



In [1009]:
# Initialize the figure
fig = go.Figure()

# Start plotting the data
for selected in range(len(linear_df)):
	# The two runs
	used_df = linear_df[selected]
	# Only plot for 512KB and 1MB, a.k.a. one fitting entirely in L1 and one not fitting
	# Plot the data
	fig.add_trace(go.Scatter(x=used_df['Processes'], 
							y=used_df['Latency'],
							mode='lines+markers',
							name="Run "+str(selected)))
# Update the layout by naming the appropirate tings and highlighting some places of interest in the x-axis
fig.update_layout(title='Latency vs Processes for the Barrier using the the Linear Algorithm', 
					xaxis_title='Processes', 
					yaxis_title='Latency (us)', 
					xaxis=dict(tickvals=tickvals)
				)

fig.show()

